In [1]:
import os
import pandas as pd
import json
import numpy as np
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from ipynb.fs.full.my_functions import factorial

ModuleNotFoundError: No module named 'ipynb.fs.full.my_functions'

In [3]:
import tweepy
import traceback

In [5]:
import spacy
import torch

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
#!pip install spacymoji
from spacymoji import Emoji

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

#!pip install transformers
from transformers import AutoModel, BertTokenizerFast

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

In [ ]:
def split_dataset(text_col, label_col):
  train_text, temp_text, train_labels, temp_labels = train_test_split(df[text_col], df[label_col], 
                                                                      random_state=2018, 
                                                                      test_size=0.3)


  val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                  random_state=2018, 
                                                                  test_size=0.5, 
                                                                  stratify=temp_labels)

  # train_frame = pd.DataFrame([train_text, train_labels])
  # val_frame = pd.DataFrame([val_text, val_labels])
  # test_frame = pd.DataFrame([test_text, test_labels])
  return train_text, train_labels, val_text, val_labels, test_text, test_labels

In [ ]:
def split_dataset_with_groups(df, text_col, label_col)

In [ ]:
print(val_labels.value_counts())
print(train_text)

0             22
1             18
unverified     5
Name: true, dtype: int64
108    PRINCE IN TORONTO TONIGHT:\n@3RDEYEGIRL tweete...
85     Up to 20 held hostage in Sydney Lindt Cafe sie...
170    Five hostages have escaped the besieged Lindt ...
76     #sydneysiege is over. 2 confirmed dead, #PrayF...
286    BREAKING: Three hostages appear to have escape...
                             ...                        
156    BREAKING  @SkyBusiness: freed hostage borne hi...
265    DETAILS: The hostage site is Lindt Chocolat Ca...
226    Currently the #FoxNews website has zero, repea...
102    Police convoy and helicopters are rushing to s...
250    Local media: 3 people appear to escape from Ma...
Name: src_text, Length: 207, dtype: object


In [ ]:
# Get features for TF-IDF
tfidf = TfidfVectorizer()#tokenizer=spacy_tokeniser)
tfidf.fit(train_text)
train_features = tfidf.transform(train_text)
validation_features = tfidf.transform(val_text)
test_features = tfidf.transform(test_text)

In [ ]:
print(train_features)

  (0, 1199)	0.24445295777678855
  (0, 1130)	0.2633723219567455
  (0, 1117)	0.3973731771501845
  (0, 1114)	0.212110091085962
  (0, 1110)	0.10389814156046775
  (0, 1094)	0.1838272529799555
  (0, 959)	0.2633723219567455
  (0, 950)	0.20491728264962708
  (0, 864)	0.19319072690600506
  (0, 799)	0.2633723219567455
  (0, 787)	0.12850499352435185
  (0, 776)	0.2633723219567455
  (0, 725)	0.22061737780894516
  (0, 584)	0.19319072690600506
  (0, 577)	0.13701228024733503
  (0, 555)	0.08130279123838846
  (0, 207)	0.2633723219567455
  (0, 156)	0.2633723219567455
  (0, 26)	0.2633723219567455
  (1, 1144)	0.25764378654454123
  (1, 1110)	0.15511401535306543
  (1, 1064)	0.18994197665972787
  (1, 995)	0.26280630051960463
  (1, 882)	0.39319989537825556
  (1, 643)	0.28842276607929185
  :	:
  (205, 507)	0.22431768606672714
  (205, 343)	0.297348050812346
  (205, 288)	0.297348050812346
  (205, 262)	0.14337508668613588
  (205, 237)	0.22431768606672714
  (205, 121)	0.17604724059817461
  (205, 110)	0.1583217914753

In [ ]:
def evaluation_summary(description, true_labels, predictions):
  print("Evaluation for: " + description)
  print(classification_report(true_labels, predictions,  digits=3, zero_division=0))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions))

In [ ]:
# SVC
svc = SVC(kernel='rbf')
svc_model = svc.fit(train_features, train_labels)
print(svc_model.score(validation_features, val_labels))
svc_predicted_labels = svc_model.predict(validation_features)

evaluation_summary("SVC", val_labels, svc_predicted_labels)
svc_test = svc_model.predict(test_features)
evaluation_summary("SVC test", test_labels, svc_test)

# Logistic Regression with TF-IDF
lr_tfidf = LogisticRegression()
lr_model_tfidf = lr_tfidf.fit(train_features, train_labels)
print(lr_model_tfidf.score(validation_features, val_labels))
print(lr_model_tfidf.score(test_features, test_labels))

lr_predicted_labels_tfidf = lr_model_tfidf.predict(validation_features)
evaluation_summary("LR (TF-IDF)", val_labels, lr_predicted_labels_tfidf)
lrtfidf_test = lr_model_tfidf.predict(test_features)
evaluation_summary("LR (TF-IDF) test", test_labels, lrtfidf_test)

# Dummy Majority
dumb = DummyClassifier(strategy='most_frequent')
dumb.fit(train_features, train_labels)
print(dumb.score(validation_features, val_labels))
dumb_validation_predicted_labels = dumb.predict(validation_features)
evaluation_summary("Dummy majority", val_labels, dumb_validation_predicted_labels)
dumb_test = dumb.predict(test_features)
evaluation_summary("Dummy MF test", test_labels, dumb_test)

0.9798766634209672
Evaluation for: SVC
              precision    recall  f1-score   support

       False      0.973     0.986     0.979     16263
        True      0.987     0.974     0.981     17628

    accuracy                          0.980     33891
   macro avg      0.980     0.980     0.980     33891
weighted avg      0.980     0.980     0.980     33891


Confusion matrix:
 [[16033   230]
 [  452 17176]]
Evaluation for: SVC test
              precision    recall  f1-score   support

       False      0.974     0.986     0.980     16264
        True      0.987     0.975     0.981     17627

    accuracy                          0.980     33891
   macro avg      0.980     0.981     0.980     33891
weighted avg      0.981     0.980     0.981     33891


Confusion matrix:
 [[16039   225]
 [  436 17191]]
0.9763359003865333
0.9760408368003305
Evaluation for: LR (TF-IDF)
              precision    recall  f1-score   support

       False      0.969     0.982     0.976     16263
     

In [ ]:
# Get output

input_text = "Watch video showing gunfire inside Canada's pa"
input_df = pd.DataFrame({"src_text": [input_text]})
input_features = tfidf.transform(input_df)

predicted_label = lr_model_tfidf.predict(input_features)

print(predicted_label)

['0']


In [ ]:
# New dataset (FakeNewsNet)
fake_df = pd.read_csv("fake.csv")
fake_df['true'] = False
real_df = pd.read_csv("real.csv")
real_df['true'] = True

df = pd.concat([fake_df, real_df])
print(len(df))
df.head()

225940


,Unnamed: 0,id,text,author_id,source,created_at,edit_history_tweet_ids,lang,description,username,verified,name,location,true
0,0,937349434668498944,BREAKING: First NFL Team Declares Bankruptcy O...,4219197432,Twitter Web Client,2017-12-03T15:54:54.000Z,['937349434668498944'],en,Ofelia. Arizmendez @ deplorable me. I am a pro...,OfeliasHeaven,False,Ofelia Duchess Arizmendez,"Sugar Land, TX",False
1,1,937379378006282240,BREAKING: First NFL Team Declares Bankruptcy O...,3018973429,Facebook,2017-12-03T17:53:54.000Z,['937379378006282240'],en,NaN,lorn_cramer,False,Lorn Cramer,NaN,False
2,2,937380068590055425,BREAKING: First NFL Team Declares Bankruptcy O...,3018973429,Facebook,2017-12-03T17:56:38.000Z,['937380068590055425'],en,NaN,lorn_cramer,False,Lorn Cramer,NaN,False
3,3,937429898670600192,BREAKING: First NFL Team Declares Bankruptcy O...,23162382,Twitter for iPhone,2017-12-03T21:14:39.000Z,['937429898670600192'],en,Happily married conservative Pentecostal woman...,starchaser57,False,🌹Star Chaser 🌼🇺🇸,Ozarks. Missouri,False
4,4,937449906352152576,BREAKING: First NFL Team Declares Bankruptcy O...,1409084934,Twitter Web Client,2017-12-03T22:34:09.000Z,['937449906352152576'],en,Deplorable member of The Silenced Majority.. #...,ThePipeStore,False,Mr. Walker 🇺🇸,NaN,False


In [ ]:
train_text, train_labels, val_text, val_labels, test_text, test_labels = split_dataset('text', 'true')

In [ ]:
print(type(train_text))
print(train_labels.value_counts())

<class 'pandas.core.series.Series'>
True     81754
False    76404
Name: true, dtype: int64


In [ ]:
# BERT

In [ ]:
# https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb#scrollTo=URn-DWJt5xhP
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

import transformers as ppb
model_class, tokenizer_class, bert_model_name = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(bert_model_name)
model = model_class.from_pretrained(bert_model_name)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = df['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
input_ids = torch.tensor(np.array(padded))

with torch.no_grad():
    last_hidden_states = model(input_ids) # ERROR: Tries to get 255 GB of RAM

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = df['true']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [ ]:
lr_clf.score(test_features, test_labels)

0.5866666666666667

In [ ]:
# Cleaning dataset

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from collections import Counter

ps = PorterStemmer()
wnl = nltk.stem.WordNetLemmatizer()

stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\karli\AppData\Roaming\nltk_data...


In [ ]:
remove_col = ['threadid','tweetid','support', 'evidentiality', 'certainty', 'event']

# Impute null values with None
def null_process(feature_df):
    col = 'src_text'
    feature_df.loc[feature_df[col].isnull(), col] = "None"
    return feature_df

# Removed unused clumns
def remove_unused_col(df,column_n=remove_col):
    df = df.drop(column_n,axis=1)
    return df

def clean_dataset(df):
    # remove unused column
    df = remove_unused_col(df)
    #impute null values
    df = null_process(df)
    return df

# Cleaning text from unused characters
def clean_text(text):
    text = str(text).replace(r'http[\w:/\.]+', ' ')  # removing urls
    text = str(text).replace(r'[^\.\w\s]', ' ')  # remove everything but characters and punctuation
    text = str(text).replace('[^a-zA-Z]', ' ')
    text = str(text).replace(r'\s\s+', ' ')
    text = text.lower().strip()
    #text = ' '.join(text)    
    return text

## Nltk Preprocessing includes:
# Stop words, Stemming and Lemmetization
def nltk_preprocess(text):
    text = clean_text(text)
    wordlist = re.sub(r'[^\w\s]', '', text).split()
    text = ' '.join([word for word in wordlist if word not in stopwords_dict])
    text = [ps.stem(word) for word in wordlist if not word in stopwords_dict]
    text = ' '.join([wnl.lemmatize(word) for word in wordlist if word not in stopwords_dict])
    return  text

In [ ]:
df = clean_dataset(df)
df["src_text"] = df.src_text.apply(nltk_preprocess)
df.head()

,true,src_text
0,unverified,vladimir putin netralized internal coup maybe ...
1,unverified,coup rt jimgeraghty rumor russian military att...
2,unverified,hoppla l0gg0l swiss rumor putin absence due gi...
3,unverified,putin reappears tv amid claim unwell threat co...
4,1,france 10 people dead shooting hq satirical we...


In [ ]:
def remove_date(row):
    return row['Created at'].timetz()

tweets_df['Created at'] = tweets_df.apply(lambda row: remove_date(row), axis=1)

tweets_df.head()

,ID,Text,Created at,User location,Num Followers,Num Friends,Num Favourites,User description,User verified,Language
0,937349434668498944,BREAKING: First NFL Team Declares Bankruptcy O...,15:54:54+00:00,"Sugar Land, TX",1712,2751,22141,Ofelia. Arizmendez @ deplorable me. I am a pro...,False,en
1,937379378006282240,BREAKING: First NFL Team Declares Bankruptcy O...,17:53:54+00:00,,14,99,125,,False,en
2,937380068590055425,BREAKING: First NFL Team Declares Bankruptcy O...,17:56:38+00:00,,14,99,125,,False,en
3,937429898670600192,BREAKING: First NFL Team Declares Bankruptcy O...,21:14:39+00:00,Ozarks. Missouri,1162,1258,9422,Happily married conservative Pentecostal woman...,False,en
4,937449906352152576,BREAKING: First NFL Team Declares Bankruptcy O...,22:34:09+00:00,,12772,13708,23094,Deplorable member of The Silenced Majority.. #...,False,en


In [ ]:
tweets_df2.to_csv("final_short.csv")

hello
